In [1]:
from packaging import version
from os import path

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

import tensorboard as tb

In [7]:
major_ver, minor_ver, _ = version.parse(tb.__version__).release
assert major_ver >= 2 and minor_ver >= 3, \
    "This notebook requires TensorBoard 2.3 or later."
print("TensorBoard version: ", tb.__version__)

TensorBoard version:  2.9.1


In [2]:
# tensorboard dev upload --logdir C:\Users\danie\OneDrive\Desktop\Bachelorarbeit\main\monitoring\logs\fit
experiment_id = "UvuVVAuXRyC0nZHwPX9fyw"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
df = experiment.get_scalars()
df.head()

,run,tag,step,value
0,cnn20220625-193909\train,epoch_accuracy,0,0.379531
1,cnn20220625-193909\train,epoch_accuracy,1,0.557656
2,cnn20220625-193909\train,epoch_accuracy,2,0.542813
3,cnn20220625-193909\train,epoch_accuracy,3,0.557344
4,cnn20220625-193909\train,epoch_accuracy,4,0.623594
